In [32]:
import pandas as pd, numpy as np, hashlib, json
from pathlib import Path

XPATH = Path("/Users/gagandeepsingh/Downloads/GSL_Case/GSL_LTV_MVP_v3.xlsx")
xls = pd.ExcelFile(XPATH)
sq   = pd.read_excel(xls, "status_quo_sample")
base = pd.read_excel(xls, "event_baselines")
cfg  = pd.read_excel(xls, "ltv_config")


In [33]:
def build_cohort_sq(
    business_model=None, country=None, platform=None, acquisition_channel=None,
    install_start=None, install_end=None
):
    df = sq.copy()
    if business_model:      df = df[df["business_model"] == business_model]
    if country:             df = df[df["country"].isin([country] if isinstance(country, str) else country)]
    if platform:            df = df[df["platform"].isin([platform] if isinstance(platform, str) else platform)]
    if acquisition_channel: df = df[df["acquisition_channel"].isin([acquisition_channel] if isinstance(acquisition_channel, str) else acquisition_channel)]
    if install_start:       df = df[df["install_date"] >= pd.to_datetime(install_start)]
    if install_end:         df = df[df["install_date"] <= pd.to_datetime(install_end)]
    return df.reset_index(drop=True)

def cohort_signature(filters: dict) -> str:
    """Stable hash from sorted JSON of filter dict."""
    norm = json.dumps(filters, sort_keys=True, default=str)
    return hashlib.sha256(norm.encode()).hexdigest()[:12]

In [34]:
def cohort_meta(df: pd.DataFrame, bm_default="F2P_IAP"):
    n = max(1, len(df))
    payer_rate = (df["totalIapRevenue"] > 0).mean()
    subs_rate  = (df["totalSubRevenue"] > 0).mean()
    ads_rate   = (df["totalAdRevenue"]  > 0).mean()
    sessions_per_user = df["totSessionCount"].mean()

    # crude D1/D7 retention proxies using lastSeenDate - install_date
    life_days = (pd.to_datetime(df["lastSeenDate"]) - pd.to_datetime(df["install_date"])).dt.days.clip(lower=0)
    d1 = (life_days >= 1).mean()
    d7 = (life_days >= 7).mean()

    bm = df["business_model"].mode().iloc[0] if len(df) else bm_default
    bm_onehot = {
        "bm_F2P_IAP": 1.0 if bm=="F2P_IAP" else 0.0,
        "bm_Subscription": 1.0 if bm=="Subscription" else 0.0,
        "bm_Ads": 1.0 if bm=="Ads" else 0.0,
        "bm_Hybrid": 1.0 if bm=="Hybrid" else 0.0,
    }
    vec = {
        "count": float(n),
        "payer_rate": float(payer_rate),
        "subs_rate":  float(subs_rate),
        "ads_rate":   float(ads_rate),
        "sessions_per_user": float(sessions_per_user),
        "ret_d1": float(d1),
        "ret_d7": float(d7),
        **bm_onehot
    }
    return pd.Series(vec)

In [35]:
def fit_two_phase_from_proxies(ret_d1, ret_d7, pivot=5):
    # simple closed-form-ish guess: solve for two churns that hit d1 and d7 roughly
    # early: days 0..pivot; late: days pivot+..7
    eps = 1e-6
    ret_d1 = max(eps, min(1.0, ret_d1))
    ret_d7 = max(eps, min(1.0, ret_d7))

    # assume late churn is milder; search a small grid
    best = (0.12, 0.03); err_best = 1e9
    for ce in np.linspace(0.05, 0.30, 26):   # early churn 5%..30%
        for cl in np.linspace(0.005, 0.08, 21):  # late churn 0.5%..8%
            s1 = (1-ce) ** 1
            s7 = (1-ce) ** min(pivot,7) * (1-cl) ** max(0,7-pivot)
            err = (s1 - ret_d1)**2 + (s7 - ret_d7)**2
            if err < err_best:
                err_best, best = err, (ce, cl)
    return {"churn_early": round(best[0],4), "churn_late": round(best[1],4), "pivot_day": int(pivot)}

In [36]:
def make_prior_row(filters):
    df = build_cohort_sq(**filters)
    sig = cohort_signature(filters)
    meta = cohort_meta(df)
    rfit = fit_two_phase_from_proxies(meta["ret_d1"], meta["ret_d7"], pivot=5)
    row = {"signature": sig, "filters": json.dumps(filters, sort_keys=True), **meta.to_dict(), **rfit}
    return row

# ... existing code ...

# EXAMPLE: build priors for a handful of common cohort filters
seed_filters = [
    {"business_model":"F2P_IAP", "country":["US","IN"], "platform":"Android", "acquisition_channel":"Facebook",
     "install_start":"2025-07-15", "install_end":"2025-07-21"},
    {"business_model":"Hybrid", "country":["IN"], "platform":"iOS", "acquisition_channel":"Organic",
     "install_start":"2025-07-01", "install_end":"2025-07-31"},
]
priors = pd.DataFrame([ make_prior_row(f) for f in seed_filters ])

# Fix: Save to same directory but as a new CSV file, not overwriting the source Excel
priors_dir = XPATH.parent  # Get the directory containing the Excel file
priors_path = priors_dir / "GSL_LTV_priors.csv"  # Create new CSV file in same directory
priors.to_csv(priors_path, index=False)
priors.head()

,signature,filters,count,payer_rate,subs_rate,ads_rate,sessions_per_user,ret_d1,ret_d7,bm_F2P_IAP,bm_Subscription,bm_Ads,bm_Hybrid,churn_early,churn_late,pivot_day
0,20c508c91772,"{""acquisition_channel"": ""Facebook"", ""business_...",2.0,0.5,0.0,0.0,17.5,1.0,0.5,1.0,0.0,0.0,0.0,0.09,0.08,5
1,ac899118deb4,"{""acquisition_channel"": ""Organic"", ""business_m...",2.0,0.0,0.0,1.0,21.5,1.0,0.0,0.0,0.0,0.0,1.0,0.25,0.08,5


In [37]:
from numpy.linalg import norm

def vec_from_row(row):
    keys = ["payer_rate","subs_rate","ads_rate","sessions_per_user","ret_d1","ret_d7",
            "bm_F2P_IAP","bm_Subscription","bm_Ads","bm_Hybrid"]
    return np.array([float(row[k]) for k in keys], dtype=float)

def cosine_sim(a,b): 
    return float(np.dot(a,b) / (norm(a)*norm(b) + 1e-12))

def lookup_or_blend(filters, priors_df, k=3):
    # 1) exact signature hit
    sig = cohort_signature(filters)
    exact = priors_df[priors_df["signature"]==sig]
    if len(exact):
        r = exact.iloc[0]
        return {"source":"cache_hit","signature":sig,
                "churn_early":r["churn_early"], "churn_late":r["churn_late"], "pivot_day":int(r["pivot_day"])}

    # 2) nearest neighbors
    df = build_cohort_sq(**filters)
    meta = cohort_meta(df)
    vq = vec_from_row(meta)

    pr = priors_df.copy()
    pr["sim"] = pr.apply(lambda r: cosine_sim(vq, vec_from_row(r)), axis=1)
    nbrs = pr.sort_values("sim", ascending=False).head(k)

    # softmax weights over similarity
    sims = nbrs["sim"].clip(lower=0).to_numpy()
    w = sims / (sims.sum() + 1e-12)

    ce = float(np.dot(w, nbrs["churn_early"]))
    cl = float(np.dot(w, nbrs["churn_late"]))
    pivot = int(round(np.dot(w, nbrs["pivot_day"])))

    return {"source":"blended_priors","signature":sig,
            "churn_early":round(ce,4), "churn_late":round(cl,4), "pivot_day":pivot}

# Example: user enters cohort filters in UI
user_filters = {"business_model":"F2P_IAP","country":["US","IN"],"platform":"Android","acquisition_channel":"Facebook",
                "install_start":"2025-07-15","install_end":"2025-07-21"}

priors_loaded = pd.read_csv(priors_path)
params = lookup_or_blend(user_filters, priors_loaded, k=2)
params

{'source': 'cache_hit',
 'signature': '20c508c91772',
 'churn_early': 0.09,
 'churn_late': 0.08,
 'pivot_day': 5}

In [38]:
def S_two_phase_from_params(days, churn_early, churn_late, pivot_day):
    t = np.array(days)
    early = np.minimum(t, pivot_day)
    late  = np.maximum(0, t-pivot_day)
    return np.clip((1-churn_early)**early * (1-churn_late)**late, 0, 1)

days = np.arange(0, 91)
S = S_two_phase_from_params(days, **{k:v for k,v in params.items() if k in ["churn_early","churn_late","pivot_day"]})

# (Optional) quick revenue toy to prove “instant” output works now:
DAU0 = len(build_cohort_sq(**user_filters))
sessions_per_1k = base[(base["event_name"]=="session_start") & 
                       (base["business_model"]==user_filters["business_model"]) &
                       (base["stage"]=="Early")]["events_per_1k_DAU_per_day_mean"].iloc[0]
actives = DAU0 * S
sessions = (actives / 1000) * sessions_per_1k
LTV_sessions_proxy_90d = sessions.sum() / max(DAU0,1)  # a toy proxy just to show end-to-end
float(LTV_sessions_proxy_90d)

1.5864705066406437